# 1. Library Imports and Data Load

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS
from statsmodels.stats.diagnostic import het_breuschpagan
from linearmodels.panel import compare
import patsy

# Load the data
df = pd.read_excel(".\\Data\\Crime_NorthCarolina.xlsx")

# Preview
print(df.head(3))


   county  year    crmrte    prbarr   prbconv   prbpris  avgsen     polpc  \
0       1    81  0.039885  0.289696  0.402062  0.472222    5.61  0.001787   
1       1    82  0.038345  0.338111  0.433005  0.506993    5.59  0.001767   
2       1    83  0.030305  0.330449  0.525703  0.479705    5.80  0.001836   

    density     taxpc  ...  lpctymle   lpctmin  clcrmrte  clprbarr  clprbcon  \
0  2.307159  25.69763  ... -2.433870  3.006608         .         .         .   
1  2.330254  24.87425  ... -2.449038  3.006608 -0.039376  0.154542  0.074143   
2  2.341801  26.45144  ... -2.464036  3.006608 -0.235316 -0.022922  0.193987   

   clprbpri  clavgsen   clpolpc   cltaxpc     clmix  
0         .         .         .         .         .  
1  0.071048 -0.003571 -0.011364 -0.032565  0.030857  
2 -0.055326  0.036879  0.038413  0.061477 -0.244732  

[3 rows x 59 columns]


# 2. Pooled OLS with Robust Standard Errors

In [2]:
# Log-transform relevant variables
df['log_crmrte'] = np.log(df['crmrte'])
df['log_prbarr'] = np.log(df['prbarr'])
df['log_prbconv'] = np.log(df['prbconv'])

# Fit pooled OLS model with robust standard errors
model = smf.ols("log_crmrte ~ log_prbarr + log_prbconv + prbpris", data=df).fit(cov_type='HC1')
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             log_crmrte   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.458
Method:                 Least Squares   F-statistic:                     75.79
Date:                Wed, 14 May 2025   Prob (F-statistic):           7.91e-42
Time:                        14:06:18   Log-Likelihood:                -347.95
No. Observations:                 630   AIC:                             703.9
Df Residuals:                     626   BIC:                             721.7
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -5.0056      0.117    -42.871      

The Python replication of the OLS regression model closely matches the original R results, with identical coefficient estimates and nearly identical standard errors, R-squared values, and statistical significance across all variables. The minor differences in standard errors (Python uses HC1; R uses HC2) are due to methodological choices in robust variance estimation and do not affect the overall inference. Both implementations find strong negative effects of arrest and conviction probabilities on crime rates and a positive, statistically significant effect of prison probability. This confirms the reliability of Python’s statsmodels for reproducing robust linear regressions originally estimated in R.

# 3. Breusch-Pagan Test

In [3]:
# Breusch-Pagan test for heteroskedasticity
residuals = model.resid
exog = model.model.exog
bp_test = het_breuschpagan(residuals, exog)

print(f"Breusch-Pagan Test: LM stat={bp_test[0]}, p-value={bp_test[1]}")


Breusch-Pagan Test: LM stat=50.568918534166876, p-value=6.044043296497876e-11


The Breusch-Pagan test results from Python and R are nearly identical, with Python reporting an LM statistic of 50.569 and a p-value of 6.044e-11, matching R’s output to three decimal places. Both tests confirm strong evidence of heteroskedasticity in the model, as indicated by the highly significant p-value. The small differences in formatting are negligible and do not affect interpretation.

# 4. Fixed and Random Effects Models

In [4]:
# Set index for panel data
df_panel = df.set_index(['county', 'year'])

# Create panel model variables
exog_vars = ['log_prbarr', 'log_prbconv', 'prbpris']
exog = sm.add_constant(df_panel[exog_vars])
y = df_panel['log_crmrte']

# Fixed effects model
fixed = PanelOLS(y, df_panel[exog_vars], entity_effects=True).fit()
print(fixed.summary)

# Random effects model
from linearmodels.panel import RandomEffects
random = RandomEffects(y, df_panel[exog_vars]).fit()


                          PanelOLS Estimation Summary                           
Dep. Variable:             log_crmrte   R-squared:                        0.0811
Estimator:                   PanelOLS   R-squared (Between):             -0.1328
No. Observations:                 630   R-squared (Within):               0.0811
Date:                Wed, May 14 2025   R-squared (Overall):             -0.1324
Time:                        14:06:18   Log-likelihood                    252.82
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      15.806
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                   F(3,537)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             15.806
                            

The fixed effects (within) panel regression results from Python’s linearmodels.PanelOLS closely replicate the output from R’s plm package. Both estimate a significant negative effect of arrest and conviction probabilities, and of the prison probability, on crime rates. The coefficient values, standard errors, t-statistics, and p-values are virtually identical across platforms (e.g., Python: log_prbarr = -0.2209, p < 0.0001; R: identical), confirming consistency in estimation. Both outputs report an R² within of ~0.0811, an F-statistic of ~15.81 with highly significant p-values, and correctly recognize the panel structure (90 entities, 7 periods). Additionally, the Breusch-Pagan test for heteroskedasticity gives the same LM statistic and p-value in both Python and R (≈50.57, p < 1e-10), further validating the equivalence of diagnostic tools.

In [5]:
print(random.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:             log_crmrte   R-squared:                        0.2386
Estimator:              RandomEffects   R-squared (Between):              0.5331
No. Observations:                 630   R-squared (Within):              -0.8197
Date:                Wed, May 14 2025   R-squared (Overall):              0.5302
Time:                        14:06:18   Log-likelihood                   -168.22
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      65.485
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                   F(3,627)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             65.485
                            

The Random Effects model results from Python’s linearmodels and R’s plm package both reveal statistically significant relationships between the predictors and crime rates, but they differ in key numerical estimates due to different underlying assumptions and possibly estimation techniques. Python reports a positive and significant effect for both log_prbarr (0.395) and log_prbconv (0.104), and a large negative effect for prbpris (−1.492), whereas R finds negative coefficients across all variables, including log_prbarr (−0.312) and log_prbconv (−0.190), and a smaller magnitude for prbpris (−0.312). Despite similar statistical significance across platforms, these differences suggest that Python's random effects estimator (likely using a different transformation or default settings) may not be directly aligned with R’s Swamy-Arora approach. Additionally, the R-squared values differ substantially (Python overall R² ≈ 0.53 vs. R ≈ 0.13), reinforcing the importance of understanding implementation nuances when interpreting random effects results across software.

# 5. Hausman Test

In [6]:
# Hausman test: compare fixed and random effects
from linearmodels.panel import compare

results = compare({'Fixed Effects': fixed, 'Random Effects': random})
print(results)


                    Model Comparison                    
                         Fixed Effects    Random Effects
--------------------------------------------------------
Dep. Variable               log_crmrte        log_crmrte
Estimator                     PanelOLS     RandomEffects
No. Observations                   630               630
Cov. Est.                   Unadjusted        Unadjusted
R-squared                       0.0811            0.2386
R-Squared (Within)              0.0811           -0.8197
R-Squared (Between)            -0.1328            0.5331
R-Squared (Overall)            -0.1324            0.5302
F-statistic                     15.806            65.485
P-value (F-stat)                0.0000            0.0000
=====================     ============   ===============
log_prbarr                     -0.2209            0.3947
                             (-5.8417)          (6.5795)
log_prbconv                    -0.1352            0.1043
                             (-

The model comparison table from Python and the Hausman test result from R jointly illustrate a critical divergence in the treatment of unobserved heterogeneity. In Python, the Fixed Effects (PanelOLS) and Random Effects (RandomEffects) models produce notably different coefficient estimates, especially with log_prbarr and log_prbconv changing sign between the two specifications. While the RE model reports higher R-squared values (overall R² = 0.5302 vs. -0.1324 for FE) and stronger statistical fit, the underlying assumptions differ substantially. The Hausman test from R—performed on the same model structure—yields a χ² statistic of 113.33 (p < 2.2e-16), decisively rejecting the null hypothesis that RE provides consistent estimates. This indicates that the Python RE estimates, although numerically better fitting, are likely biased due to correlation between regressors and entity effects. Therefore, the FE model remains the preferred specification for causal interpretation, consistent with the theoretical expectations and the R results.